<a href="https://colab.research.google.com/github/Swaroop0724/Commercial-water-vending-machine-using-Arduino/blob/main/Server_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers fastapi uvicorn python-multipart pyngrok -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.9 MB/s eta 0:00:00


In [2]:
import logging
from fastapi import FastAPI, Request, HTTPException
import uvicorn
import torch
import nest_asyncio
from google.colab import userdata
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM

logging.basicConfig(level=logging.DEBUG)

# Load the Chatbot Model
model_name = "gpt2"
# model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create the FastAPI App
app = FastAPI()

# Define an Endpoint for Chatbot Interaction
@app.post("/chat/")
async def chat(request: Request):
    try:
        data = await request.json()
        user_input = data["message"]
        history = data.get("history", [])

        # Construct the input prompt
        prompt = f"You are a concise chatbot. User: {user_input} Chatbot Response: "
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        # Generate the response
        output = model.generate(input_ids, max_length=500, pad_token_id=tokenizer.eos_token_id)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Update history
        new_history = history + [{"user": user_input, "bot": bot_response}]

        logging.debug(f"User Input: {user_input}")
        logging.debug(f"Bot Response: {bot_response}")
        logging.debug(f"History: {new_history}")

        return {"response": bot_response, "history": new_history}

    except Exception as e:
        logging.exception("Exception in /chat/ endpoint:")
        raise HTTPException(status_code=500, detail="Internal server error")

# Run the FastAPI App and Expose it with Ngrok
if __name__ == "__main__":
    nest_asyncio.apply()

    authtoken = userdata.get("NGROK_KEY")

    if authtoken:
        ngrok.set_auth_token(authtoken)

        http_tunnel = ngrok.connect(8000)
        print("Public URL:", http_tunnel.public_url)

        uvicorn.run(app, host="0.0.0.0", port=8000)
    else:
        print("Error: ngrok authtoken not found. Please add it to Colab user secrets as 'NGROK_KEY'.")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

ERROR:pyngrok.process.ngrok:t=2025-04-04T20:20:03+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-04T20:20:03+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.